Hi! Much of the computer vision code was generated with ChatGPT in an effot to expedite the more tedious aspects of this project. I will try to outline which parts are GPT vs my own.

In [5]:
# Holy moly everything below created by GPT

import cv2
import tkinter as tk
import random
from PIL import Image, ImageTk
import math

class App:
    def __init__(self, window):
        self.window = window
        self.window.title("Camera + Game")

        # --- Layout Frames ---
        self.left_frame = tk.Frame(window)
        self.left_frame.pack(side=tk.LEFT)

        self.right_frame = tk.Frame(window)
        self.right_frame.pack(side=tk.RIGHT)

        # --- Camera Setup ---
        self.cap = cv2.VideoCapture(0)
        self.out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*'XVID'), 20.0, (640, 480))

        self.video_label = tk.Label(self.left_frame)
        self.video_label.pack()
        self.update_camera()

        # --- Game Canvas ---
        self.canvas_width = 400
        self.canvas_height = 400
        self.canvas = tk.Canvas(self.right_frame, width=self.canvas_width, height=self.canvas_height, bg="black")
        self.canvas.pack()

        # --- Player Circle (Green) ---
        self.circle_radius = 20
        self.circle_x = self.canvas_width // 2
        self.circle_y = self.canvas_height // 2
        self.circle = self.canvas.create_oval(
            self.circle_x - self.circle_radius, self.circle_y - self.circle_radius,
            self.circle_x + self.circle_radius, self.circle_y + self.circle_radius,
            fill="green"
        )

        # --- Goal Circle (Red) ---
        self.goal_radius = 15
        self.goal = None
        self.spawn_goal_near(self.circle_x, self.circle_y)

        # --- Score ---
        self.score = 0
        self.score_label = tk.Label(self.right_frame, text=f"Score: {self.score}", font=("Helvetica", 16), fg="white", bg="black")
        self.score_label.pack(pady=10)

        # --- Key Bindings ---
        self.window.bind("<KeyPress-w>", lambda e: self.move_circle(0, -10))
        self.window.bind("<KeyPress-s>", lambda e: self.move_circle(0, 10))
        self.window.bind("<KeyPress-a>", lambda e: self.move_circle(-10, 0))
        self.window.bind("<KeyPress-d>", lambda e: self.move_circle(10, 0))

        # --- Quit Button ---
        self.quit_button = tk.Button(self.right_frame, text="Quit", command=self.quit)
        self.quit_button.pack(pady=10)

    def update_camera(self):
        ret, frame = self.cap.read()
        if ret:
            self.out.write(frame)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            self.video_label.imgtk = imgtk
            self.video_label.configure(image=imgtk)
        self.window.after(10, self.update_camera)

    def move_circle(self, dx, dy):
        new_x = self.circle_x + dx
        new_y = self.circle_y + dy

        # Boundary check
        if 0 + self.circle_radius <= new_x <= self.canvas_width - self.circle_radius:
            self.circle_x = new_x
        if 0 + self.circle_radius <= new_y <= self.canvas_height - self.circle_radius:
            self.circle_y = new_y

        self.canvas.coords(
            self.circle,
            self.circle_x - self.circle_radius, self.circle_y - self.circle_radius,
            self.circle_x + self.circle_radius, self.circle_y + self.circle_radius
        )

        self.check_collision()

    def check_collision(self):
        gx1, gy1, gx2, gy2 = self.canvas.coords(self.goal)
        goal_cx = (gx1 + gx2) / 2
        goal_cy = (gy1 + gy2) / 2

        dist = math.hypot(goal_cx - self.circle_x, goal_cy - self.circle_y)
        if dist < self.circle_radius + self.goal_radius:
            self.score += 1
            self.score_label.config(text=f"Score: {self.score}")
            self.canvas.delete(self.goal)
            self.spawn_goal_near(self.circle_x, self.circle_y)

    def spawn_goal_near(self, x, y):
        range_min = -100
        range_max = 100
        while True:
            dx = random.randint(range_min, range_max)
            dy = random.randint(range_min, range_max)
            new_x = x + dx
            new_y = y + dy
            if self.goal_within_bounds(new_x, new_y):
                break

        self.goal = self.canvas.create_oval(
            new_x - self.goal_radius, new_y - self.goal_radius,
            new_x + self.goal_radius, new_y + self.goal_radius,
            fill="red"
        )

    def goal_within_bounds(self, x, y):
        return (
            self.goal_radius <= x <= self.canvas_width - self.goal_radius and
            self.goal_radius <= y <= self.canvas_height - self.goal_radius
        )

    def quit(self):
        self.cap.release()
        self.out.release()
        self.window.destroy()

# Run it!
root = tk.Tk()
app = App(root)
root.mainloop()
